# Rename files for input4mips

In [1]:
from datetime import datetime
import netCDF4 as nc
import os

In [2]:
infolder_list = [
    "../output/aerosols/", 
    "../output/aviation/", 
    "../output/GHGs/"
]
outfolder = "../output/input4mips/"
versions_list = [
    "v4.8.1",
    "v4.8",
    "v4"
]
print_version = "v4.8.1"
assert len(versions_list) == len(infolder_list)

In [3]:
target_mip = "CovidMIP"
old_long_scen_name = "IAMC-MESSAGE-GLOBIOM_SSP2-45-SPA2"
for ind in range(len(infolder_list)):
    infolder = infolder_list[ind]
    version = versions_list[ind]
    files = os.listdir(infolder)
    files = [file for file in files if (file.__contains__(version))]

    for file in files:
        compress = True
        newfilename = file[4:].split(".nc")[0]
        if infolder.__contains__("aerosols"):
            scenario = file.split(".nc_")[1]
            scenario = scenario[:-3]
        elif infolder.__contains__("aviation"):
            scenario = file.split("_" + version)[0].split("_")[-1]
            newfilename = newfilename.replace(scenario, "")
            if scenario.__contains__("4year"):
                scenario = "FourYearBlip"
            elif scenario.__contains__("baseline"):
                scenario = "baseline"
            elif scenario.__contains__("mp06"):
                scenario = "TwoYearBlip"
            else:
                raise ValueError
        elif infolder.__contains__("GHGs"):
            scenario = file.replace("_" + version, "").split(".nc")[1]
        if file.__contains__("baseline") | file.__contains__("Base"):
            print("Skip file {}".format(file))
            continue
        elif scenario == "TwoYearBlip":
            scenario_label = "ssp245-covid"
        elif scenario == "FourYearBlip":
            scenario_label = "ssp245-covid4y"
        elif scenario == "ModerateGreen":
            scenario_label = "ssp245-cov-modgreen"
        elif scenario == "FossilFuel":
            scenario_label = "ssp245-cov-fossil"
        elif scenario == "StrongGreen":
            scenario_label = "ssp245-cov-strgreen"
        else:
            print("Cannot find the name for {}".format(scenario))
            raise ValueError
        newfilename = newfilename.replace(scenario + "_", "").replace("emissions_ScenarioMIP", target_mip).replace(
            "GHGConcentrations_ScenarioMIP", target_mip
        ).replace(
            "-1-1", "").replace("-1-2-1", "").replace("IAMC-", "").replace("UoM-", ""
        ).replace(
            "MESSAGE-GLOBIOM-ssp245", "Imperial_" + scenario_label + "_" + print_version
        ).replace("4year_", "").replace("201501-210012", "201501-205012") + ".nc"
        print(scenario)
        print(newfilename)

        ## Copy the previous file details

        db = nc.Dataset(os.path.join(infolder, file))
        trg = nc.Dataset(os.path.join(outfolder, newfilename), mode="w")
        # Create the dimensions of the file
        for name, dim in db.dimensions.items():
            trg.createDimension(
                name, len(dim) if not (dim.isunlimited()) and not (name == "time") else None
            )

        # Copy the global attributes
        trg.setncatts({a: db.getncattr(a) for a in db.ncattrs()})

        for name, var in db.variables.items():
            if compress:
                trg.createVariable(
                    name, var.dtype, var.dimensions, complevel=9, shuffle=True, zlib=True
                )
            else:
                trg.createVariable(name, var.dtype, var.dimensions)

            # Copy the variable attributes
            trg.variables[name].setncatts({a: var.getncattr(a) for a in var.ncattrs()})

            # Copy the variables values, removing some times if needed
            if "time" in var.dimensions[:]:
                trg.variables[name][:] = db.variables[name][:]


        # Rewrite the meta variables
        trg.comment = db.comment.replace(
            old_long_scen_name, scenario + ", based on " + old_long_scen_name + 
            " modified by the impacts of lockdown for coronavirus"
        )
        trg.contact = "Robin D. Lamboll (r.lamboll@imperial.ac.uk)"
        trg.creation_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
        trg.institution = "Imperial College London"
        trg.institution_id = "ImperialCollege"
        trg.references = "https://doi.org/10.5194/gmd-2020-373"
        trg.source = "https://zenodo.org/record/3947917#.YK0Pc6hKhPZ"
        trg.source_id = "Zenodo"
        trg.source_version = version
        trg.target_mip = "CovidMIP"
        db.close()
        trg.close()

Skip file cut_NOx-em-AIR-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc_baseline4year_v4.8.nc
Skip file cut_NOx-em-AIR-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc_baseline_v4.8nc
FourYearBlip
NOx-em-AIR-anthro_input4MIPs_CovidMIP_Imperial_ssp245-covid4y_v4.8.1_gn_201501-205012.nc


C:\Users\rlamboll\AppData\Local\Continuum\anaconda3\envs\netcdf-5\lib\site-packages\ipykernel_launcher.py:65: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\rlamboll\AppData\Local\Continuum\anaconda3\envs\netcdf-5\lib\site-packages\ipykernel_launcher.py:76: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\rlamboll\AppData\Local\Continuum\anaconda3\envs\netcdf-5\lib\site-packages\ipykernel_launcher.py:84: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


TwoYearBlip
NOx-em-AIR-anthro_input4MIPs_CovidMIP_Imperial_ssp245-covid_v4.8.1_gn_201501-205012.nc
